In [3]:
pwd

'c:\\Users\\USER\\AppData\\Local\\Programs\\Microsoft VS Code'

In [1]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")
[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

c:\Users\USER\anaconda3\envs\key_bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [3]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# KoBERT 모델과 토크나이저 불러오기
model = BertModel.from_pretrained('skt/kobert-base-v1')

# 문서 예제
text = "여기에 분석하고자 하는 문서를 넣으세요. 예를 들어, 이 문장은 BERT 모델을 사용하여 키워드를 추출하는 예제입니다."

# 토큰화 및 토큰 ID로 변환
input_ids = tokenizer.encode(text, add_special_tokens=True)
input_ids_tensor = torch.tensor([input_ids])

# 모델을 사용하여 출력 계산
with torch.no_grad():
    outputs = model(input_ids_tensor)

# 어텐션 스코어 추출
attention = outputs.attentions
last_layer_attention = attention[-1]
mean_attention = last_layer_attention.mean(dim=1)
token_attention_scores = mean_attention.squeeze().mean(dim=0).numpy()

# TF-IDF 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform([text])
tfidf_scores = np.array(tfidf.sum(axis=0)).flatten()

# 단어 사전 구축
feature_names = tfidf_vectorizer.get_feature_names_out()

# 어텐션 스코어와 TF-IDF 스코어 결합
combined_scores = token_attention_scores * tfidf_scores

# 결합된 스코어를 기반으로 상위 키워드 추출
num_keywords = 5
top_indices = combined_scores.argsort()[-num_keywords:]
keywords = [feature_names[i] for i in top_indices]

print("Extracted Keywords:", keywords)


TypeError: 'NoneType' object is not subscriptable